In [22]:
import pygame as pg
import numpy as np
import math
import random

def map_range(val, min_from, max_from, min_to, max_to):
    val -= min_from
    val /= (max_from-min_from)
    val *= (max_to-min_to)
    val += min_to
    return val

target_points = []

class Sinfunc:
    def __init__(self):
        self.coefficients = []
        for c in [1, 1, 0, 0, 0, 1, 0, 0]:
            self.coefficients.append(c)
        self.derivatives = []
        self.lr = 0.1

    def eval(self, x):
        sx = self.coefficients[0]
        sy = self.coefficients[1]
        tx = self.coefficients[2]
        ty = self.coefficients[3]
        x *= sx
        x += tx
        result = 0
        coeffs_per_iteration = 4
        for coefficient_index in range(4, len(self.coefficients)-coeffs_per_iteration):
            inner_shift = self.coefficients[coefficient_index]
            inner_scale = self.coefficients[coefficient_index+1]
            outer_scale = self.coefficients[coefficient_index+2]
            outer_shift = self.coefficients[coefficient_index+3]

            result += math.cos(x*math.pi*2*inner_scale + inner_shift)*outer_scale + outer_shift

        for coefficient in self.coefficients:
            result += math.sin(x*coefficient*math.pi)
        return result*sy + ty

    def approximate(self, targets):
        learning_rate = self.lr
        dx = 0.000001
        self.derivatives = []
        error_sum = 0
        for coefficient_index, coefficient in enumerate(self.coefficients):
            total_change = 0
            for target in targets:
                tx, ty = target
                y0 = self.error(ty, self.eval(tx))
                self.coefficients[coefficient_index] += dx
                y1 = self.error(ty, self.eval(tx))
                self.coefficients[coefficient_index] = coefficient
                dydc = (y1-y0)/dx
                total_change+= dydc
            if abs(total_change) > 100:
                total_change = 1
                self.lr *= 0.8
            self.derivatives.append(total_change)
        for coefficient_index, coefficient in enumerate(self.coefficients):
            self.coefficients[coefficient_index] -= self.derivatives[coefficient_index]*learning_rate

    def error(self, val, expected_val):
        if abs(expected_val-val) > 2**10:
            return 2**10
        return 0.5*(expected_val - val)**2

bg_col = (0, 0, 0)
pg.init()
width, height = (800, 800)
screen = pg.display.set_mode((width, height))
clock = pg.time.Clock()
running = True

func = Sinfunc()

while running:
    mouse_x, mouse_y = pg.mouse.get_pos()
    mouse_x = map_range(mouse_x, 0, width, -1, 1)
    mouse_y = map_range(mouse_y, height, 0, -1, 1)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_ESCAPE:
                running = False
            if event.key == pg.K_a:
                func.lr /= 1.05
                print(func.lr)
            if event.key == pg.K_d:
                func.lr *= 1.05
                print(func.lr)
        if event.type == pg.MOUSEBUTTONDOWN:
            if event.button == 1:
                target_points.append((mouse_x, mouse_y))
            else:
                for c in [0, 1, 2, 0]:
                    func.coefficients.append(c)
            func.approximate(target_points)
    
    
    
    screen.fill(bg_col)
    for point in target_points:
        px, py = point
        px = map_range(px, -1, 1, 0, width)
        py = map_range(py, -1, 1, height, 0)
        pg.draw.circle(screen, (125, 125, 255), (px, py), 5, 0)

    
    for x in np.arange(-1, 1, 0.01):
        y = func.eval(x)
        point = (map_range(x, -1, 1, 0, width), map_range(y, -1, 1, height, 0))
        pg.draw.circle(screen, (255, 255, 255), (point), 1, 0)

    pg.display.flip()
    clock.tick(30)
pg.quit()




0.09523809523809523
0.09070294784580497
0.08638375985314758
0.08227024747918817
0.07835261664684587
0.07462153966366272
0.07106813301301211
0.06768393620286868
0.06446089162177969
0.06139132535407589
6.217376636874525e-06
5.921311082737643e-06
5.6393438883215645e-06
5.370803703163394e-06
5.115051145869899e-06
